In [ ]:
import time
import json
from jira import JIRA


options ={
    'server':'https://issues.apache.org/jira/'
}

project_name = "ActiveMQ"

jira = JIRA(options)
sess_get = jira._session.get
projects = jira.projects()


for p in projects:
    if p.name == project_name:
        project = p
        print(project.name)
        break
            
issues = []            


keepCrawling = True
while keepCrawling:
    tmp = jira.search_issues('project='+project.name+ ' AND status=Resolved AND resolution=Fixed', startAt=0, maxResults=100)
    print('.', end="")
    if (len(tmp) > 0):
        issues.append(tmp)
    else:
        keepCrawling = False

#Jira on Apache has a limitation of 100
print(len(issues))



ActiveMQ
..............................

In [10]:
for issue in issues:
    print(issue.id) 
    DEV_STATUS = 'https://issues.apache.org/jira/rest/dev-status/1.0'
    _issue = 'issue/detail?issueId=%s' % issue.id
    _args = 'applicationType=fecru&dataType=repository&_=%s' % int(time.time())
    req_url = '%s/%s&%s' % (DEV_STATUS, _issue, _args)
    response = sess_get(req_url)
    raw_data = json.loads(response.content.decode('utf-8'))
    
    try:
        hasCommit = True
        commits = raw_data['detail'][0]['repositories'][0]['commits']
    except IndexError:
        hasCommit = False
    if hasCommit:
        for req in commits:
            #print(req)
            print('%s\n%s\n\n' % (req['displayId'], req['files']))

    
#print(issues)

#issue = jira.issue('JRA-9')
#print(issue.fields.project.key)             # 'JRA'
#print(issue.fields.issuetype.name)          # 'New Feature'
#print(issue.fields.reporter.displayName)  # 'Mike Cannon-Brookes [Atlassian]'

12955093
98165c4
[{'path': 'activemq-amqp/src/main/java/org/apache/activemq/transport/amqp/AmqpFrameParser.java', 'url': 'https://fisheye6.atlassian.com/browse/activemq-git/activemq-amqp/src/main/java/org/apache/activemq/transport/amqp/AmqpFrameParser.java?r1=61a3eab8ab4a1280c285229830820dea4db1d7f7&r2=98165c4b69f0a5df93a7c964b9b9e143c7f7a083', 'linesRemoved': 1, 'linesAdded': 2, 'changeType': 'MODIFIED'}, {'path': 'activemq-client/src/main/java/org/apache/activemq/openwire/OpenWireFormat.java', 'url': 'https://fisheye6.atlassian.com/browse/activemq-git/activemq-client/src/main/java/org/apache/activemq/openwire/OpenWireFormat.java?r1=13044deccea1e1bd6849dbe0babb6720aed634ac&r2=98165c4b69f0a5df93a7c964b9b9e143c7f7a083', 'linesRemoved': 2, 'linesAdded': 3, 'changeType': 'MODIFIED'}, {'path': 'activemq-client/src/main/java/org/apache/activemq/util/IOExceptionSupport.java', 'url': 'https://fisheye6.atlassian.com/browse/activemq-git/activemq-client/src/main/java/org/apache/activemq/util/IOE